In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Here are the descriptions for some of the columns:

* name - The name of the country.
* area - The total land and sea area of the country.
* population - The country's population.
* population_growth- The country's population growth as a percentage.
* birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
* death_rate - The country's death rate, or the number of death a year per 1,000 * people.
* area- The country's total area (both land and water).
* area_land - The country's land area in square kilometers.
* area_water - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and look for any outlier countries.

In [5]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [6]:
%%sql
SELECT
    MIN(population),
    MAX(population),
    MIN(population_growth),
    MAX(population_growth)
    
FROM
    facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

* there's a country with a population of 0
* there's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

In [10]:
%%sql
SELECT
    name, population
FROM
    facts
WHERE population = (SELECT MIN(population) FROM facts)
    OR population = (SELECT MAX(population) FROM facts);

Done.


name,population
Antarctica,0
World,7256490011


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for Antarctica

Let's continue by calculating some averages.

In [12]:
%%sql
SELECT
    AVG(population) AS average_population,
    AVG(area) AS average_area
FROM
    facts;

Done.


average_population,average_area
62094928.32231405,555093.546184739


To finish, we'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have:

* Above average values for population.
* Below average values for area.

In [19]:
%%sql
SELECT
    name,
    population,
    area
FROM
    facts
WHERE
    population > (SELECT AVG(population) FROM facts)
    AND area < (SELECT AVG(area) FROM facts)
ORDER BY
    name;

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


Here are some next steps for you to explore:

* Which countries have the highest ratios of water to land? Which countries have more water than land?

In [20]:
%%sql
SELECT
    name,
    CAST(area_water AS float) / CAST(area_land AS float) AS water_land_ratio
FROM
    facts
ORDER BY
    water_land_ratio DESC
LIMIT 10;

Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


In [21]:
%%sql
SELECT
    name,
    area_water,
    area_land
FROM
    facts
WHERE
    area_water > area_land
LIMIT 10;

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


Which countries will add the most people to their population next year?

In [26]:
%%sql
SELECT
    name,
    population,
    CAST(population * (population_growth / 100) AS int) AS added_people
FROM
    facts
ORDER BY
    added_people DESC
LIMIT 5;

Done.


name,population,added_people
World,7256490011,78370092
India,1251695584,15270686
China,1367485388,6153684
Nigeria,181562056,4448270
Pakistan,199085847,2906653


Which countries have a higher death rate than birth rate?

In [28]:
%%sql
SELECT
    name,
    birth_rate,
    death_rate
FROM
    facts
WHERE
    death_rate > birth_rate;

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73
